In [1]:
print("Type your image url here: ")
url = input()

Do you want to use a url or your own file? [U/f]
f


Saving butt.jpg to butt.jpg


In [2]:
#install dependencies: (use cu101 because colab has CUDA 10.1)
!pip3 install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip3 install cython pyyaml==5.1
!pip3 install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.5 in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision==0.6 in /usr/local/lib/python3.6/dist-packages (0.6.0+cu101)
     |████████████████████████████████| 276kB 9.1MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=732deb96040f222df42d048c7353add1c001b84ab8e18b1ee82b17ce70d5c139
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-ezvjmynj
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-ezvjmynj
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-lin

In [3]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2MB 3.1MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200607-cp36-none-any.whl size=41028 sha256=9b81c98c91784be7bc4069a242b4cb9a2c58b90432fa97e92dc02b3eaca6c40d
  Stored in directory: /root/.cache/pip/wheels/5f/d6/68/b36c8d0b4a602e5c86b4010142aa7744b46c1556afd29544bc
Successfully built fvcore


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

This is where we get images, we'll replace that url with our own


In [0]:
try:
  !wget $url -O HTNE_image.jpg
  im = cv2.imread("./HTNE_image.jpg")
  cv2_imshow(im)
except:
  !wget "https://www.wedaways.com/wp-content/uploads/2019/10/hotel-room-blocks.jpg" -O HTNE_image.jpg
  im = cv2.imread("./HTNE_image.jpg")
  cv2_imshow(im)

In [6]:

cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

model_final_f10217.pkl: 178MB [00:17, 9.95MB/s]                           


this removes the prediction boxes from the images

In [0]:
print(outputs["instances"].pred_boxes)
outputs["instances"].pred_boxes = torch.tensor([np.zeros(4) for i in range(outputs["instances"].pred_boxes.__len__())], device='cuda:0')
print(outputs["instances"].pred_boxes)

v.get_image() is our output

In [0]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2, instance_mode=detectron2.utils.visualizer.ColorMode.IMAGE_BW)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
image_final = v.get_image()[:, :, ::-1]
cv2_imshow(image_final)

In [0]:
cv2.imwrite("HTNE_stage.jpg", image_final)
cv2.imwrite("HTNE_image.jpg", im)
files.download("HTNE_image.jpg")
files.download("HTNE_stage.jpg")